In [ ]:
# Download Vietnamese stopword here: https://github.com/stopwords/vietnamese-stopwords
# Dowload Vncorenlp here: https://github.com/vncorenlp/VnCoreNLP

In [ ]:
import pandas as pd
import re
import string
from vncorenlp import VnCoreNLP
!pip install networkx
!pip install googletrans
from googletrans import Translator
import networkx as nx
from itertools import combinations
import matplotlib.pyplot as plt
import nltk
from nltk import FreqDist
from collections import Counter

In [ ]:
import pandas as pd
# !pip install vncorenlp
data = pd.read_excel('insert content data here.xlsx',index_col=None,na_values=['NA'],usecols="B")
sample_data = data[:]

In [ ]:
len(sample_data)

In [ ]:
import io
stop_words = [line.strip() for line in open('vietnamese-stopwords.txt','r', encoding ="utf8")]
stop_words = set([w.lower().replace(' ','_') for w in stop_words])

In [ ]:
annotator = VnCoreNLP("vncore_nlp/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx4g')

In [ ]:
WEB_ADDRESS_RE = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

In [ ]:
def get_clean_sentences(content):
    content = re.sub(WEB_ADDRESS_RE, ' ', content)
    content = content.strip().split('\n')
    content = [string for string in content if string != ""]
    clean_sentences = []
    for p in content:
        sentences = annotator.annotate(p)['sentences']
        for s in sentences:
            words = [word['form'].lower() for word in s if word['posTag'].startswith('N')]
            words = [w for w in words if w not in string.punctuation and w not in stop_words]
            clean_sentences.append(words)
    return clean_sentences

In [ ]:
# clean data
start=datetime.now()
clean_sentences=[]
for _, row in sample_data.iterrows():
    clean_sentence = get_clean_sentences(row['content'])
    clean_sentences.append(clean_sentence)
clean_sentences
print(datetime.now()-start)

In [ ]:
# make list
start=datetime.now()
words =[]
for sentences in clean_sentences:
    for sentence in sentences:
        words.append(sentence)
print(datetime.now()-start)

In [ ]:
#define most common word
from datetime import datetime
start=datetime.now()
common=[]
for sentence in words:
    for word in sentence:
        common.append(word)
most_common = Counter(common).most_common(50)
print(datetime.now()-start)
# set(list(dict(most_common).keys()))
print(most_common)

In [ ]:
for j in most_common:
    print(j)

In [ ]:
#count #co-occurences
co_occurence_graph = {}
start=datetime.now()
for word in words:
    token_pairs = combinations(word,2)
    for u, v in token_pairs:
        if u in set(list(dict(most_common).keys())) and v in set(list(dict(most_common).keys())):
            if u != v:
                if u in co_occurence_graph:
                    if v in co_occurence_graph[u]:
                        co_occurence_graph[u][v]['weight'] += 1
                    else:
                        co_occurence_graph[u][v] = {'weight':1}
                else:
                    co_occurence_graph[u] = {v:{'weight':1}}
print(datetime.now()-start)

In [ ]:
# convert to networkx graph                    
co_occurence_graph = nx.from_dict_of_dicts(co_occurence_graph)

In [ ]:
# run k-core procedure
g = nx.k_core(co_occurence_graph,k=5)

In [ ]:
#export to gephi input
nx.write_gexf(co_occurence_graph, "period_all.gexf")

In [ ]:
# export nodes
nodes=[]
for node in co_occurence_graph.nodes:
    nodes.append(node)
nodes_all = open("nodes.txt", "w",encoding="utf8")
for element in nodes:
    nodes_all.write(element+'\n')
nodes_all.close()

In [ ]:
# export edges
nx.write_edgelist(co_occurence_graph, "edges_all.txt", delimiter=' ', data=['weight'])